## FAIMX

Here we're using the following video as an example:<br>
**TolucaVSPumas_sample.mp4**

In [0]:
import pickle
import cv2
import os
import numpy as np
from scipy.signal import savgol_filter

In [0]:
video = "drive/My Drive/data/faimx/TolucaVSPumas_sample.mp4"

### 1. Object Detection using Detectron2

The idea is to detect teams and referees.<br>
See **Detectron2.ipynb** for details.<br>
The output is a pickle file (list) with all the detections for each frame on a video.

In [0]:
team_detections_file = "drive/My Drive/data/faimx/team_detections.pickle"
with open(team_detections_file, 'rb') as fp:
    frames_detections = pickle.load(fp)

print('Total Frames: {}'.format(len(frames_detections)))
for i, frame_detections in enumerate(frames_detections):
    print(frame_detections)
    if i == 8:
        break

### 2. Run IOU Tracker
Here we want to run the tracker because we want to smooth each player trajectory, otherwise there's too much jitter and movement. <br>
See **scripts/iou_tracker.py** for more details. <br>
The input are the predictions from above while the output is anoter pickle file (list) with all the tracks associated for each detection. 

In [0]:
tracks_file = "drive/My Drive/data/faimx/tracks.pickle"
with open(tracks_file, 'rb') as fp:
    tracks = pickle.load(fp)

print('Total Tracks: {}'.format(len(tracks)))
for i, track in enumerate(tracks):
    print(track)
    if i == 1:
        break

Optionally, you can use the functions **visualize_video()** and **plot_graph()** to visualize the intermediate results.

### 3. Convert to a 2D map representation.  
Before running this, you need to make sure you have the necessary Homography matrices to transform the images. Otherwise it won't be possible. <br>
See **Pitch.ipynb** at the end for more details.

The input are the Original Tracks while the output is another pickle file (list) with the Map Tracks.

Remember that they are different because they have different X,Y coordinates. So, basically the Map Tracks represent the transformation of coordinates from the Original Tracks using Homography matrices. 


In [0]:
map_tracks_file = "drive/My Drive/data/faimx/map_tracks.pickle"
with open(map_tracks_file, 'rb') as fp:
    map_tracks = pickle.load(fp)

print('Total Tracks: {}'.format(len(map_tracks)))
for i, map_track in enumerate(map_tracks):
    print(map_track)
    if i == 1:
        break

Optionally, you can use the function **visualize_pitch()** and **plot_graph()** to visualize the intermediate results.

### 4. Run Pose Estimation

See **Gluon-cv_PoseEstimation.ipynb** for details.

The Detectron2 detections or the IOU Tracks are used as input (pickle file) and the Output is the video with the Pose Estimations drawn.

### 4. Visualizing results

Here we are just adding the pitch over the videos generated before. 

In [0]:
## Utils

def resize(image, width=None, height=None, inter=cv2.INTER_AREA):
    # initialize the dimensions of the image to be resized and
    # grab the image size
    dim = None
    (h, w) = image.shape[:2]

    # if both the width and height are None, then return the
    # original image
    if width is None and height is None:
        return image

    if width and height:
        dim = (width, height)
    elif width is None:
        # calculate the ratio of the height and construct the dimensions
        r = height / float(h)
        dim = (int(w * r), height)
    elif height is None:
        # calculate the ratio of the width and construct the dimensions
        r = width / float(w)
        dim = (width, int(h * r))

    # resize the image
    resized = cv2.resize(image, dim, interpolation=inter)

    # return the resized image
    return resized

In [10]:
input_video_filename = 'drive/My Drive/data/faimx/pose_estimation.mp4'
input_video = cv2.VideoCapture(input_video_filename)
num_frames = int(input_video.get(cv2.CAP_PROP_FRAME_COUNT))
width = int(input_video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(input_video.get(cv2.CAP_PROP_FRAME_HEIGHT))
frames_per_second = input_video.get(cv2.CAP_PROP_FPS)
num_frames = int(input_video.get(cv2.CAP_PROP_FRAME_COUNT))

output_video_filename = 'drive/My Drive/data/faimx/faimx.mp4'
output_video = cv2.VideoWriter(filename=output_video_filename, fourcc=cv2.VideoWriter_fourcc(*'mp4v'), fps=float(frames_per_second), frameSize=(width, height), isColor=True)

## Smooth data
data = []
for t in map_tracks:
    ids = []
    x_avgs = []
    y_avgs = []
    frame_numbers = []
    scores = []
    class_ids = []
    print(t['id'], len(t['bbox']))
    for bbox in t['bbox']:

        ids.append(t['id'])
        x_avgs.append(bbox[-1][0][0])
        y_avgs.append(bbox[-1][1][0])
        frame_numbers.append(bbox[0])
        scores.append(bbox[1])
        class_ids.append(bbox[2])

    data.append({
        'ids': ids,
        'x_avgs': x_avgs,
        'y_avgs': y_avgs,
        'frame_numbers': frame_numbers,
        'scores': scores,
        'class_ids': class_ids
    })

    for d in data:
        # print(set(d['ids']), len(d['x_avgs']), len(d['y_avgs']))
        assert len(d['x_avgs']) == len(d['y_avgs'])
        if len(d['x_avgs']) > 400:
            d['x_avgs_smooth'] = savgol_filter(d['x_avgs'], 199, 3)
            d['y_avgs_smooth'] = savgol_filter(d['y_avgs'], 199, 3)
        elif len(d['x_avgs']) > 200:
            d['x_avgs_smooth'] = savgol_filter(d['x_avgs'], 99, 3)
            d['y_avgs_smooth'] = savgol_filter(d['y_avgs'], 99, 3)
        elif len(d['x_avgs']) > 100:
            d['x_avgs_smooth'] = savgol_filter(d['x_avgs'], 49, 3)
            d['y_avgs_smooth'] = savgol_filter(d['y_avgs'], 49, 3)
        elif len(d['x_avgs']) > 50:
            d['x_avgs_smooth'] = savgol_filter(d['x_avgs'], 23, 3)
            d['y_avgs_smooth'] = savgol_filter(d['y_avgs'], 23, 3)
        elif len(d['x_avgs']) > 25:
            d['x_avgs_smooth'] = savgol_filter(d['x_avgs'], 11, 3)
            d['y_avgs_smooth'] = savgol_filter(d['y_avgs'], 11, 3)
        elif len(d['x_avgs']) > 12:
            d['x_avgs_smooth'] = savgol_filter(d['x_avgs'], 5, 3)
            d['y_avgs_smooth'] = savgol_filter(d['y_avgs'], 5, 3)
        else:
            d['x_avgs_smooth'] = d['x_avgs']
            d['y_avgs_smooth'] = d['y_avgs']
        
## END Smooth data

bbox_colors = [(255, 255, 255), (235, 208, 52), (8, 0, 255)]
i = 0
while input_video.isOpened():

    if i % 60 == 0:
        prog = round(i / num_frames, 2) * 100
        print('{}%'.format(prog))

    ret, frame = input_video.read()

    if ret:

        pitch = cv2.imread('drive/My Drive/data/faimx/pitch.png')

        for d in data:
            for fid, frame_number, x_avg, y_avg, score, class_id in zip(d['ids'], d['frame_numbers'], d['x_avgs_smooth'], d['y_avgs_smooth'], d['scores'], d['class_ids']):
                if fid in [30, 31, 32, 33, 47, 50]:
                    continue
                if frame_number == i:
                    bbox_color = bbox_colors[class_id]
                    cv2.circle(pitch, (int(x_avg), int(y_avg)), 10, bbox_color, -1)

        pitch = resize(pitch, height=210)
        x_position = frame.shape[0] - pitch.shape[0] - 10 ## Around bottom of image
        y_position = int((frame.shape[1] - pitch.shape[1]) / 2) ## Around center of image
        blend = 0.7

        rows, cols, channels = pitch.shape
        pitch = cv2.addWeighted(frame[x_position:x_position+rows, y_position:y_position+cols], blend, pitch, blend, 0)
        frame[x_position:x_position+rows, y_position:y_position+cols] = pitch

        # cv2.imwrite('/home/alex/Downloads/pitch/result/{}.jpg'.format(i), frame)

        output_video.write(frame)
        i += 1

    else:
        break

input_video.release()
output_video.release()

1 588
2 358
3 653
4 366
5 359
6 181
7 642
8 642
9 617
10 490
11 96
12 384
13 413
14 852
15 2
16 520
17 368
18 1
19 244
20 404
21 1
22 407
23 547
24 392
25 206
26 299
27 122
28 154
29 64
30 6
31 48
32 1
33 18
34 340
35 122
36 73
37 62
38 294
39 3
40 288
41 273
42 269
43 10
44 13
45 194
46 182
47 5
48 3
49 6
50 8
51 1
52 6
0.0%
5.0%
10.0%
15.0%
20.0%
25.0%
30.0%
35.0%
40.0%
45.0%
50.0%
55.00000000000001%
60.0%
65.0%
71.0%
76.0%
81.0%
86.0%
91.0%
96.0%


Optionally, use the Homography matrix inverse to draw lines on broadcast image